In [27]:
import random

f = open("dataset_865384_5.txt","r")

k, t = [int(x) for x in next(f).split()]
Dna = [str(x) for x in next(f).split()]


#k, t = 8, 5
#Dna = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA", "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", "TAGTACCGAGACCGAAAGAAGTATACAGGCGT", "TAGATCAAGTTTCAGGTGCACGTCGGTGAACCAA", "TCCACCAGCTCCACGTGCAATGTTGGCCTA"]

PSEUDO_OFFSET = 4
RANDOM_ITERATIONS = 1000

def RandSearchXTimes(Dna, k, t, iterations):
    BestMotifs = RandomizedMotifSearch(Dna, k, t)
    BestScore = Score(BestMotifs)
    
    for i in range(iterations):
        motifs = RandomizedMotifSearch(Dna, k, t)
        score = Score(BestMotifs)
        
        if Score(motifs) < BestScore:
            BestMotifs = motifs
            BestScore = Score(BestMotifs)
    
    return BestMotifs

def RandomizedMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(len(Dna)):
        r = random.randint(0, len(Dna[i]) - k)
        BestMotifs.append(Dna[i][r:r + k])
    
    while True:
        profile = FindProfile(BestMotifs, k)
        motifs = Motifs(profile, Dna)
        if Score(motifs) < Score(BestMotifs):
            BestMotifs = motifs
        else:
            break
    return BestMotifs    
    
        
def FindProfile(motifs, k):
    Profile = []
    for i in range(k):
        Profile.append({"A":1.0,"C":1.0,"G":1.0,"T":1.0})
    for motif in motifs:
        for nucleotide in range(len(motif)):
            Profile[nucleotide][motif[nucleotide]] += 1.0
    for i in range(k):
        for key in Profile[i]:
            Profile[i][key] /= (len(motifs) + PSEUDO_OFFSET)
            
    return Profile
    
def Motifs(Profile, Dna):
    motifs = []
    for Text in Dna:
        motifs.append(ProfileMostProbableKmer(Text, len(Profile), Profile))
    return motifs
    
def ProfileMostProbableKmer(Text, k, Profile):
    MostProbableKmer = ""
    KmerProbability = 0
    
    for i in range(len(Text) - k + 1):
        kmer = Text[i:i + k]
        prob = 1
        for n in range(len(kmer)):
            prob = prob * Profile[n][kmer[n]]
            
        if prob > KmerProbability:
            KmerProbability = prob
            MostProbableKmer = kmer
            
    return MostProbableKmer

def Score(motifs):
    Profile = []
    k = len(motifs[0])
    for j in range(k):
        Profile.append({"A":0.0,"C":0.0,"G":0.0,"T":0.0})
    for motif in motifs:
        for nucleotide in range(len(motif)):
            Profile[nucleotide][motif[nucleotide]] += 1.0
    
    score = 0
    for i in range(k):
        max_key = "A"
        max_val = float('-inf')
        for key in Profile[i]:
            if Profile[i][key] > max_val:
                max_val = Profile[i][key]
                max_key = key
        
        for key in Profile[i]:
            if key != max_key:
                score += Profile[i][key]
                
    return score
  
out = ""
for motif in RandSearchXTimes(Dna, k, t, RANDOM_ITERATIONS):
    out = out + motif + " "
print(out)

f.close()

TAGTTCGAAGAAGGT GGGAATCAAGATGGA GGGGTACAAGAGTCA GGGGTACAATTAGGA GGGGTAGCCGATGGA GGGGCTAAAGATGGA GGGTACCAAGATGGA GGGGTCACAGATGGA GGGGTACAAGATTTT AAGGTACAAGATGGT GGGGTACATACTGGA CGGGTACAAGATGAT GGGGTACAAGTGTGA GGGGTATCCGATGGA GGGGTACTCTATGGA GTTTTACAAGATGGA CTAGTACAAGATGGA GGGGTTGGAGATGGA GGTCAACAAGATGGA GGGGGCGAAGATGGA 
